# Identification of Kossakowski Markovian ME
## (for spin-boson system using TSSOS POP method)  

In [1]:
include("LiPoSID.jl")

using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra

using HDF5
using DynamicPolynomials

using Dates

using Statistics

using TSSOS

In this notebook we identify and test accuracy of Markovian master equations in Kossakowski form for the nontrivial example of the data generated by the full-featured simulation of the spin-boson model (two-level system coupled with the bath of the continuum of the harmonic oscillators).  


According to the theorem proven by Kossakowski (Kossakowski, 1973) Markovian master equation could be rewritten for a two-level system $\rho \in M(2)$ as follows:

$
   \mathcal{L} = -i[H,\rho] + \frac{1}{2} \sum_{i,j}^3 C_{ij} \{[f_i, \rho f_j] + [f_i\rho, f_j]\}, 
$
where

$
\text{(i)} \quad \quad 
           f_i f_j = \frac{1}{4}\delta_{ij}I  +\frac{i}{2} \sum_{k=1}^3 \epsilon_{ijk} f_k \\ \text{which means:} \notag \\ \quad
    \operatorname{Tr}(f_i f_j)=\frac{1}{2}\delta_{ij}, \quad \operatorname{Tr}(f_i) = 0 \notag 
$


and we can choose basis operators $f_i$ as halfs of Pauli matrices 
$\{ \sigma_x, \sigma_y, \sigma_z \}$:

$
    f_1 = \sigma_x/2  = 
    \begin{pmatrix}
        0 & 1 \\ 1 & 0
    \end{pmatrix} / 2 \\
    f_2 = \sigma_y/2  = \begin{pmatrix}
        0 & i \\ -i & 0
    \end{pmatrix} / 2\\
    f_3 = \sigma_z/2 = \begin{pmatrix}
        1 & 0 \\ 0 & -1
    \end{pmatrix} / 2
$

In [2]:
σˣ = [ 0 1 
       1 0 ]

σʸ = [ 0.   im*1
      -im*1 0    ]

σᶻ = [ 1.  0
       0  -1 ] 

fᴷ₁ = σˣ/2; fᴷ₂ = σʸ/2; fᴷ₃ = σᶻ/2

@assert tr(σˣ/2*σʸ/2) == tr(σˣ/2*σᶻ/2) ==  tr(σʸ/2*σᶻ/2) ≈ 0
@assert tr(σˣ/2*σˣ/2) == tr(σʸ/2*σʸ/2) == tr(σᶻ/2*σᶻ/2) ≈ 1/2

fᴷᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

3-element Vector{Matrix{ComplexF64}}:
 [0.0 + 0.0im 0.5 + 0.0im; 0.5 + 0.0im 0.0 + 0.0im]
 [0.0 + 0.0im 0.0 + 0.5im; 0.0 - 0.5im 0.0 + 0.0im]
 [0.5 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -0.5 + 0.0im]

$
     \text{(ii)} \quad \quad \quad 
     H = \sum_1^3 h_i f_i, \quad h_i \in \mathbb{R}
$

which means we can use Hamiltonian of the form:

$
H = \begin{pmatrix} 
h_3   & h_1 + i h_2          \\
h_1 - i h_2 & -h_3
\end{pmatrix} / 2 
$

In [3]:
@polyvar ϵ h₁ h₂ 

Hˢʸᵐᵇ = [ ϵ           h₁+im*h₂
           h₁-im*h₂   -ϵ            ] / 2

Hᴷˢʸᵐᵇ = h₁ * fᴷ₁ + h₂ * fᴷ₂  + ϵ * fᴷ₃ 

@assert tr(Hᴷˢʸᵐᵇ) == 0
@assert Hᴷˢʸᵐᵇ == Hˢʸᵐᵇ

$
     \text{(iii)} \quad \{C_{ij}\} = \begin{pmatrix} 
\gamma - 2\gamma_1    & - i a_3            &  i a_2 \\
i a_3                 & \gamma - 2\gamma_2 & -i a_1 \\
-i a_2                & i a_1              & \gamma - 2\gamma_3 
\end{pmatrix} 
=  \begin{pmatrix} 
\kappa_1   & - i a_3    &  i a_2 \\
i a_3       & \kappa_2   & -i a_1 \\
-i a_2      & i a_1      & \kappa_3 
\end{pmatrix}
$

In [4]:
@polyvar κ[1:3]; κ₁,κ₂,κ₃ = κ[1],κ[2],κ[3]
@polyvar a[1:3]; a₁,a₂,a₃ = a[1],a[2],a[3]

Cˢʸᵐᵇ = [  κ[1]     -im*a[3]     im*a[2]
           im*a[3]   κ[2]       -im*a[1] 
          -im*a[2]   im*a[1]     κ[3]     ] ;

$
    \text{(iv)} \quad \quad \quad \quad \gamma_1, \gamma_2, \gamma_3 \geq 0
$


If $\gamma = 0$ the evolution is just unitary, and in case $\gamma >0 $ <br>
in order for the evolution to be completely positive it is nessesary and sufficient that <br>
(denoting $\kappa_i = \gamma - 2 \gamma_i$):

$
    \kappa_1 + \kappa_2 + \kappa_3 \geq 0 \\
\kappa_1 \kappa_2  + \kappa_3 \kappa_1  + \kappa_1 \kappa_2  \geq a_1^2 + a_2^2 + a_3^2 \\
   \kappa_1 \kappa_2 \kappa_3 \geq  \kappa_1 a_1^2 + \kappa_2 a_2^2 + \kappa_3 a_3^2 
$

In [5]:
constr1 = κ₁ + κ₂ + κ₃ 
constr2 = κ₁*κ₂ + κ₃*κ₁ + κ₂*κ₃ - - a₁^2 - a₂^2 - a₃^2
constr3 = κ₁*κ₂*κ₃ - κ₁*a₁^2 - κ₂*a₂^2 - κ₃*a₃^2

constraints = [ ϵ, κ₁, κ₂, κ₃, constr1, constr2, constr3] 

7-element Vector{Polynomial{true, Int64}}:
 ϵ
 κ₁
 κ₂
 κ₃
 κ₁ + κ₂ + κ₃
 κ₁κ₂ + κ₁κ₃ + κ₂κ₃ + a₁² - a₂² - a₃²
 κ₁κ₂κ₃ - κ₁a₁² - κ₂a₂² - κ₃a₃²

To ensure the objective is polynomial we can write loss function with double step and using the Frobenius norm :

$ L = \sum_{j=3}^N
{\left\| \rho_j - \rho_{j-2} - {\mathcal{L}} \left[\int^{t_j}_{t_{j-2}}\rho(t)dt \right] \right\| }^2_F $

and apply the Simpson approximation method to evaluate the integral:

$
\int^{t_j}_{t_{j-2}}\rho(t)dt = \frac{1}{3} \Delta t \left[ \rho_{j-2} + 4 \rho_{j-1} + \rho_j \right] +  \mathcal{O}(\Delta t^5).$

Using it we can calculate objective as polynomial of nine variables  and cast the optimization problem:

$
\min L\left( h_1, h_2, h_3,\kappa_1,\kappa_2, \kappa_3, a_1, a_2, a_3, \{\rho_k\} \right)
$

which is subject to one linear and two polynomial constraints above. 

In this way we obtain constraint polynomial optimization problem that can be easily and naturally solved with TSSOS library.

Below is the function `kossak_obj()`that implements assembling of the objective from the time series of density matrices $\{\rho_k\}$ at moments of time $\{\rho_i\}$. 

In [6]:
function kossak_obj(ρ, t, Hˢʸᵐᵇ, Cˢʸᵐᵇ, Fᴼᴺᴮ)

    function Dc(ρ, t)
        U = (Hˢʸᵐᵇ*ρ - ρ*Hˢʸᵐᵇ)/im 
        D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
        return U + D
    end 

    obj = 0
    for i in 3:length(ρ)
        obj += LiPoSID.frobenius_norm2(
            ρ[i] - ρ[i-2] - (t[i]-t[i-1])*(Dc(ρ[i], t[i])+
            4*Dc(ρ[i-1], t[i-1])+Dc(ρ[i-2], t[i-2]))/3
        )
    end

    if isempty(monomials(obj))
        obj = 0. 
    else
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    end

    return obj

end

kossak_obj (generic function with 1 method)

Density matrices for four bases initial states $\ket{0}$, $\ket{1}$, $\ket{x}$, $\ket{y}$ that were used to simulate training data are defined as follows:
\begin{align}
   \rho^{(g)}_0 &=  \rho^{\ket{0}}_0 = \left(
\begin{array}{c}
            1  \\
            0   \\
        \end{array}
                \right) 
                \left(
                \begin{array} {cc}
                1 & 0
                \end{array}
                \right) = 
\ket{0} \bra{0} = \left(
        \begin{array}{ccccc}
            1  &  0  \\
            0  &  0   \\
        \end{array}
            \right),\\
\rho^{(e)}_0 &=\rho^{\ket{1}}_0 =\ket{1} \bra{1} = 
\begin{pmatrix}
0 \\ 1
\end{pmatrix}
\begin{pmatrix}
0 & 1
\end{pmatrix}=
\left(
        \begin{array}{ccccc}
            0  &  0  \\
            0  &  1  \\
        \end{array}
            \right), \notag\\
\rho^{\ket{x}}_0 &=\ket{x} \bra{x} = |+\rangle \langle + | = \left(
\begin{array}{c}
            1/\sqrt{2}  \\
            1/\sqrt{2}   \\
        \end{array}
                \right) 
                \left(
                \begin{array} {cc}
                1/\sqrt{2} & 1/\sqrt{2} 
                \end{array}
                \right) = \frac{1}{2}\left(
        \begin{array}{ccccc}
            1  &  1  \\
            1  &  1   \\
        \end{array}
    \right), \notag\\
\rho^{\ket{y}}_0  &= \ket{y} \bra{y} = |-\rangle \langle-| = \left(
\begin{array}{c}
            1/\sqrt{2}  \\
            i/\sqrt{2}   \\
        \end{array}
                \right) 
                \left(
                \begin{array} {cc}
                1/\sqrt{2} & -i/\sqrt{2} 
                \end{array}
                \right) = \frac{1}{2}\left(
        \begin{array}{ccccc}
            1  &  -i  \\
            i  &  1   \\
        \end{array}
    \right)
\end{align}

The function `kossak_GEXY_obj()` 
below combines the objective for the for data series of density matrices starting in four basis states 
$\ket{0}$, $\ket{1}$, $\ket{x}$, $\ket{y}$:

In [7]:
function kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_obj(ρᵍ, tᵍ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyE = kossak_obj(ρᵉ, tᵉ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyX = kossak_obj(ρˣ, tˣ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyY = kossak_obj(ρʸ, tʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end

kossak_GEXY_obj (generic function with 1 method)

### References

1. Kossakowski, Bull. Acad. Pol. Sci. Ser. Math. Astr. Phys. 21, 649 (1973)

2. Completely positive dynamical semigroups of Nlevel systems 
Vittorio Gorini, Andrzej Kossakowski, and E. C. G. Sudarshan
Citation: J. Math. Phys. 17, 821 (1976); doi: 10.1063/1.522979

In [8]:
# Here we set parameters what data to use for training and testing

evol_data_file_name = "DATA/ALL_GAMMAS_B4_D10.h5"

tests_dir = ""

ρᵍ₀ = [ 1 0
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];
basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states

all_states = vcat(train_states, test_states);

Checking functions on one particular coupling level

In [9]:
γᵢ = "0.25133"

tᵍᵉˣʸ , ρᵍᵉˣʸ  = LiPoSID.read_GEXY_timeevolution(evol_data_file_name, γᵢ)
polyGEXYfull = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

0.10341163322607885ϵ² - 4.104630886766538e-5ϵh₁ + 3.451755329501592e-5ϵh₂ - 0.0002589838118092574ϵκ₁ + 0.0002589838118092574ϵκ₂ + 0.0012450858989678402ϵa₁ - 0.0008044950814777315ϵa₂ + 1.0519096548020683h₁² + 0.0005179676236185148h₁h₂ + 5.432584669112943e-20h₁κ₁ - 1.725877664750785e-5h₁κ₂ + 1.725877664750796e-5h₁κ₃ - 2.1131584754297306h₁a₂ - 0.0012450858989678404h₁a₃ + 1.0518715157602614h₂² - 2.052315443383269e-5h₂κ₁ - 3.3868128414917834e-20h₂κ₂ + 2.052315443383269e-5h₂κ₃ + 2.1131584754297306h₂a₁ + 0.0008044950814777305h₂a₃ + 0.2629774137005171κ₁² + 0.5000923843340622κ₁κ₂ + 0.025862443066971434κ₁κ₃ - 0.0006225429494839201κ₁a₂ + 1.0565792377148653κ₁a₃ + 0.26296787894006535κ₂² + 0.02584337354606811κ₂κ₃ - 0.00040224754073886574κ₂a₁ + 1.0565792377148653κ₂a₃ + 0.025852908306519713κ₃² - 0.00040224754073886574κ₃a₁ - 0.0006225429494839201κ₃a₂ + 1.269688888901564a₁² - 8.005395128533376e-37a₁a₂ - 4.658513032068184e-37a₁a₃ + 1.269688888901564a₂² + 4.647304221135261e-36a₂a₃ + 1.269688888901564a₃² -

In [10]:
variables(polyGEXYfull)'

1×9 adjoint(::Vector{PolyVar{true}}) with eltype PolyVar{true}:
 ϵ  h₁  h₂  κ₁  κ₂  κ₃  a₁  a₂  a₃

In [11]:
LiPoSID.coefficient_range(polyGEXYfull)

7.125837652874591e-39

Below is the wrapper function to call the TSSOS library function for the constraint polynomial optimization:

In [12]:
function min_cs_tssos(p, constrs)

    coeffs = coefficients(p)
    reg_coef = 0

    pop =[p+reg_coef*sum(variables(p).^2), constrs...] ./ maximum(abs.(coeffs))

    d = maxdegree(p)
    
    # Initial optimization step
    opt, sol, data = cs_tssos_first(pop, variables(pop), d; solution=true, QUIET=true)
    ref_sol, flag = TSSOS.refine_sol(opt, sol, data; QUIET=true)
    prev_opt, prev_sol, prev_data = opt, sol, data 

    # Check if the solution needs further refinement
    if flag != 0
        while ~isnothing(sol) && flag != 0
            prev_opt, prev_sol, prev_data = opt, sol, data
            opt, sol, data = cs_tssos_higher!(data; solution=true, QUIET=true) 
        end
        ref_sol, flag = TSSOS.refine_sol(prev_opt, prev_sol, prev_data; QUIET=true)
    end

    solution = variables(p) => ref_sol

    if flag == 0 
        status_name = "GLOBAL"
    else
        status_name = "LOCAL/FAIL"
    end

    return solution, status_name

end

min_cs_tssos (generic function with 1 method)

In [13]:
sol, st = min_cs_tssos(polyGEXYfull, constraints)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[8, 7]
[1, 1]
-----------------------------------------------------------------------------
optimum = 4.694092738736601e-6
Global optimality certified with relative optimality gap 0.000032%!

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Global optimality certified with relative optimality gap 0.000951%!


(PolyVar{true}[ϵ, h₁, h₂, κ₁, κ₂, κ₃, a₁, a₂, a₃] => [25.140907362543857, 0.000276938268486056, -0.00016636574463716887, 0.3583550700151173, 0.18237860441852494, 0.0010986066520666368, 2.3612851882529155e-5, 1.8519677711058513e-6, -0.2568046278558424], "GLOBAL")

In [14]:
Cˢⁱᵈ = convert.(ComplexF64,subs(Cˢʸᵐᵇ, sol))
Cˢⁱᵈ[1:2,1:2]

2×2 Matrix{ComplexF64}:
 0.358355+0.0im            0.0+0.256805im
      0.0-0.256805im  0.182379+0.0im

In [15]:
sol, st = min_cs_tssos(LiPoSID.filter_odd_terms_by_relative_threshold(polyGEXYfull, 1e-4), constraints)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[6, 2, 1]
[1, 2, 1]
-----------------------------------------------------------------------------
optimum = 6.238006900811503e-8
Global optimality certified with relative optimality gap 0.000011%!
Global optimality certified with relative optimality gap 0.001181%!


(PolyVar{true}[ϵ, h₁, h₂, κ₁, κ₂, κ₃, a₁, a₂, a₃] => [25.140687057273993, -0.026750789681977468, -0.040394886607506224, 0.25022355504390975, 0.2577643397234901, 0.08926101924096891, 0.04021490209924806, -0.02663256378282678, -0.2406725416178681], "GLOBAL")

In [16]:
st

"GLOBAL"

In [17]:
Cˢⁱᵈ = convert.(ComplexF64,subs(Cˢʸᵐᵇ, sol))
Cˢⁱᵈ[1:2,1:2]

2×2 Matrix{ComplexF64}:
 0.250224+0.0im            0.0+0.240673im
      0.0-0.240673im  0.257764+0.0im

In [18]:
Hˢⁱᵈ = convert.(ComplexF64, subs(Hˢʸᵐᵇ, sol))

2×2 Matrix{ComplexF64}:
    12.5703+0.0im        -0.0133754-0.0201974im
 -0.0133754+0.0201974im    -12.5703+0.0im

Below we perform system identification and testing of the accuracy of identified models for all available coupling levels of spin boson system:

In [26]:
println(" SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI Frobenius objective QO simulation")

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

evol_data_file_name = "DATA/ALL_GAMMAS_B4_D10.h5"

relative_threshold = 1e-15

rltrs = string(convert(Int, floor(log10(relative_threshold))))

tests_data_file_name = "KOSSAK_CONSTR_TSSOS_treshold_1e"*rltrs*"_FROB_QO_"*date_and_time_string * ".h5"

FminGammas = []
FmedianGammas = []
FmeanGammas = []
Epsilons = []
CoefRanges = []

for γᵢ in γ

    println("γ =  "*γᵢ)

    tᵍᵉˣʸ , ρᵍᵉˣʸ  = LiPoSID.read_GEXY_timeevolution(evol_data_file_name, γᵢ)

    elapsed_time = @timed begin

        polyGEXYfull = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)     
        #polyGEXY = polyGEXYfull
        polyGEXY = LiPoSID.filter_odd_terms_by_relative_threshold(polyGEXYfull, relative_threshold)
        sol, status = min_cs_tssos(polyGEXY, constraints)

    end

    @show minimum(abs.(coefficients(polyGEXY)))
    @show maximum(abs.(coefficients(polyGEXY)))
    push!(CoefRanges, LiPoSID.coefficient_range(polyGEXYfull))

    print(" status:", status)
    print(" runtime :", elapsed_time.time)

    Hˢⁱᵈ = convert.(ComplexF64, subs(Hˢʸᵐᵇ, sol))
    Cˢⁱᵈ = convert.(ComplexF64,subs(Cˢʸᵐᵇ, sol))
    epsilon = subs(ϵ, sol)

    push!(Epsilons, epsilon)
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, γᵢ) # create gamma coupling group   
        γ_group["epsilon"] = convert(Float64, epsilon)
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        γ_group["C"] = convert.(ComplexF64, Cˢⁱᵈ)
        γ_group["status"] = status
        γ_group["runtime"] = elapsed_time.time

    end

    println()

    FminStates = []
    FmedianStates = []
    FmeanStates = []

    for state in test_states # loop over initial states
        
        print(state*" ")

        start_time = time()

        tₛ, ρₛ = LiPoSID.read_timeevolution(evol_data_file_name, state, γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        ρₒ = DenseOperator(basis, ρₛ[1])
        dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
        tᵉⁿᵈ = tᵗˢᵗ[end]

        #print("effective_Lindblad_ops for Kossakowski")

        effective_Lindblad = LiPoSID.get_lindblad_operators(convert.(ComplexF64, Cˢⁱᵈ), fᴷᴼᴺᴮ)
        effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

        #print("Simulating Kossakowski")

        tout, ρ_t_kossak = timeevolution.master(convert.(Float64, tᵗˢᵗ), ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
        ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

        #bˢⁱᵈ = LiPoSID.bloch(ρˢⁱᵈ)
        
        # Calculating fidelity
        
        F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)


        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, γᵢ) # open gamma coupling group
            init_state_group = create_group(γ_group, state) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, F)
            #init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
            #init_state_group["bloch_sid"] = convert.(Float64, bˢⁱᵈ)
            init_state_group["tr_dist_grnd"] = LiPoSID.TrDist(ρₛ[1], ρᵍ₀)
            init_state_group["time"] = tᵗˢᵗ
        end
        
        FminState = minimum(F)
        FmedianState = mean(F)
        FmeanState = mean(F)
        
        push!(FminStates, FminState)
        push!(FmedianStates, FmedianState)
        push!(FmeanStates, FmeanState)
    
    end

    # Calculate the mean
    F_mean_value = mean(FmeanStates)

    # Calculate the median
    F_median_value = median(FmedianStates)

    # Calculate the min
    F_min_value = minimum(FminStates)

    push!(FminGammas, F_min_value)
    push!(FmedianGammas, F_median_value)
    push!(FmeanGammas, F_mean_value)

    println("Median fidelity for "*γᵢ*": ", F_median_value)

end

h5open(tests_dir*tests_data_file_name,"cw") do fid
    fid["F_min"] = convert.(Float64,FminGammas)
    fid["F_median"] = convert.(Float64,FmedianGammas)
    fid["F_mean"] = convert.(Float64,FmeanGammas)
    fid["Energy"] = convert.(Float64,Epsilons)
    fid["CoefsRanges"] = convert.(Float64,CoefRanges)

end


println(tests_data_file_name)

 SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI Frobenius objective QO simulation
γ =  0.079477
*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[8, 7]
[1, 1]
-----------------------------------------------------------------------------
optimum = 6.902585156696847e-7
Global optimality certified with relative optimality gap 0.000024%!
Global optimality certified with relative optimality gap 0.000011%!
minimum(abs.(coefficients(polyGEXY))) = 5.358499781758037e-5
maximum(abs.(coefficients(polyGEXY))) = 501.8627949039175
 status:GLOBAL runtime :3.308350342
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9960366903168857
γ =  0.25133
*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------


In [27]:
println(tests_data_file_name)

KOSSAK_CONSTR_TSSOS_treshold_1e-15_FROB_QO_2024-Sep-06_at_11-57.h5
